In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
import collections

import numpy as np
import tensorflow as tf
import tensorflow_federated as tff

np.random.seed(0)

In [4]:
emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data()

In [5]:
from tensorflow_federated.proto.v0 import computation_pb2 as pb


NUM_CLIENTS = 10
NUM_EPOCHS = 5
SHUFFLE_BUFFER = 100

def preprocess(dataset):

  def map_fn(element):
    return collections.OrderedDict(
        x=tf.reshape(element['pixels'], [-1, 784]),
        y=tf.reshape(element['label'], [-1, 1]))

  return dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER, seed=1).map(map_fn)

In [6]:
example_dataset = emnist_train.create_tf_dataset_for_client(
    emnist_train.client_ids[0])
preprocessed_example_dataset = preprocess(example_dataset)

In [7]:
def create_keras_model():
  return tf.keras.models.Sequential([
      tf.keras.layers.InputLayer(input_shape=(784,)),
      tf.keras.layers.Dense(10, kernel_initializer='zeros'),
      tf.keras.layers.Softmax(),
  ])
  
def model_fn():
  keras_model = create_keras_model()
  return tff.learning.from_keras_model(
      keras_model,
      input_spec=preprocessed_example_dataset.element_spec,
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [8]:
model = create_keras_model()
input_spec = preprocessed_example_dataset.element_spec
functional_model = tff.learning.models.functional_model_from_keras(keras_model=model, loss_fn=tf.keras.losses.SparseCategoricalCrossentropy(),input_spec=input_spec)
functional_model

In [9]:
iterative_process = tff.learning.build_federated_averaging_process(
    lambda: functional_model,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0))

TypeError: Expected tensorflow_federated.python.learning.model.Model or keras.engine.training.Model, found tensorflow_federated.python.learning.models.functional.FunctionalModel.

In [10]:
new_model = tff.learning.models.model_from_functional(functional_model)
iterative_process = tff.learning.build_federated_averaging_process(
    lambda: new_model,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0))

AttributeError: in user code:

    File "/home/teo/anaconda3/envs/PySyTFF/lib/python3.9/site-packages/tensorflow_federated/python/learning/optimizers/keras_optimizer.py", line 66, in mock_apply_gradients  *
        opt.apply_gradients([
    File "/home/teo/anaconda3/envs/PySyTFF/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/optimizer_v2.py", line 678, in apply_gradients  **
        return tf.__internal__.distribute.interim.maybe_merge_call(
    File "/home/teo/anaconda3/envs/PySyTFF/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/optimizer_v2.py", line 722, in _distributed_apply
        var.op.name):

    AttributeError: Tensor.op is undefined when eager execution is enabled.


In [11]:
new_model = tff.learning.models.model_from_functional(functional_model)
iterative_process = tff.learning.algorithms.build_unweighted_fed_avg(
    lambda: new_model,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0))

RuntimeError: resource: Attempting to capture an EagerTensor without building a function.

In [12]:
iterative_process = tff.learning.algorithms.build_unweighted_fed_avg(
    lambda: functional_model,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0))

TypeError: Expected tensorflow_federated.python.learning.model.Model or keras.engine.training.Model, found tensorflow_federated.python.learning.models.functional.FunctionalModel.